In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mido import MidiFile, tick2second
from pretty_midi import PrettyMIDI
import pickle
from sklearn.externals import joblib
import subprocess
import os
import os.path
import random
from ExtractBootlegFeatures1 import *
import multiprocessing

/home/dyang/.conda/envs/MIR4/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


### Extract Bootleg Features from Images

In [3]:
def convert_Pdf_wrapper(i):
    basedir = 'experiments/allBootlegScores-new/'
    outdir = '{}/scores/'.format(basedir)
    imagedir = '/data1/dyang/Datasets/5000dataset/images1/'
    numPdfs=5024;
    if i >= numPdfs:
        return
    if not os.path.isdir(outdir):
        os.makedirs(outdir)
    
    outfile = outdir + 'p{}.pkl'.format(str(i))
    if i <= 200:
        filepath = imagedir + 'p'+str(i)+'/'
    else:
        filepath = imagedir + 'p'+str(i)+'/'
    if os.path.exists(outfile):
        print(str(i)+" is already there")
        return
    total_bscore = np.zeros((62,1))
    lst = len(os.listdir(filepath))
    if lst == 1:
        if i<=200:
            imagepath = filepath+'p{}.jpg'.format(i)
        else:
            imagepath = filepath+'p{}.jpg'.format(i)
        try:
            bscore_query = processImageFile(imagepath,None)
        except:
            bscore_query = np.zeros((62,1))
        total_bscore = np.concatenate((total_bscore, bscore_query),axis=1)
    else:
        for count in range(lst):
            try:
                if i<=200:
                    imagepath = filepath+'p{}-{}.jpg'.format(i,count)
                else:
                    imagepath = filepath+'p{}-{}.jpg'.format(i,count)
                bscore_query=processImageFile(imagepath,None)
            except:
                bscore_query = np.zeros((62,1))
            total_bscore = np.concatenate((total_bscore, bscore_query),axis=1)
    with open(outfile, 'wb') as f:
        print("Saving p" + str(i))
        pickle.dump(total_bscore, f)
    print(total_bscore.shape)
        #visualizeLongBootlegScore(total_bscore, [13,15,17,19,21,35,37,39,41,43])

In [4]:
convert_Pdf_wrapper(23)

FileNotFoundError: [Errno 2] No such file or directory: '/data1/dyang/Datasets/5000dataset/images1/p23/'

In [21]:
inputs = [[i] for i in range(1,4825)]
n_cores = 28

pool = multiprocessing.Pool(processes=n_cores)
outputs = list(pool.starmap(convert_Pdf_wrapper, inputs))

# Construct Reverse Index

In [5]:
def bootlegHash(arr):
    bitstring = ""
    for i in range(len(arr)):
        if arr[i]==1:
            bitstring+="1"
        else:
            bitstring +="0"
    bitstring = bitstring+"00"
    hashint = int(bitstring, 2)
    hashint = np.uint64(hashint)
    return hashint

In [6]:
def getTotalBscore(bscore_file):
    bscore_array = []
    with open (bscore_file,'rb') as f:
        bscore_array = pickle.load(f)
    f.close()
    total_bscore = np.array([]).reshape(62,0)
    page_array = []
    for page in bscore_array:
        total_page = np.array([]).reshape(62,0)
        for num in page:
            col = np.array(decodeColumn(num)).reshape(62,-1)
            total_page = np.concatenate((total_page,col),axis=1)
        total_bscore = np.concatenate((total_bscore,total_page),axis=1)
        page_array.append(total_page)
    return total_bscore,page_array

In [7]:
def decodeColumn(num):
    col = []
    for i in range(62):
        col.insert(0,num%2)
        num = int(num/2)
    return col

In [8]:
def getRandomdb(size, mode, seed = 1):
    # Generates db.list
    f = open("cfg_files/db.list", "w")
    random.seed(seed)
    randomDir = []
    if(mode == "all"):
        directory = 'experiments/bootleg_output/'
        for root, dirs, files in os.walk(directory):
            for file in files:
                fname = os.path.join(root,file)
                if fname[-4:] == ".pkl":
                    num = fname.split('/')[-1]
                    str2 = 'experiments/bootleg_output/'+num+"\n"
                    f.write(str2)
        f.close()    
        return
    if size > 200:
        randomDir = random.sample(range(201, 36834), size-200)
    print(randomDir)
    for i in range(1,201):
        num = str(i)
        str2 = 'experiments/bootleg_output/'+"p"+num+".pkl"+"\n"
        f.write(str2)
        
    for i in randomDir:
        num = str(i)
        str2 = 'experiments/bootleg_output/'+"p"+num+".pkl"+"\n"
        f.write(str2)
    f.close()

In [9]:
getRandomdb(200,"all")

In [10]:
def Singular_DB(data, rindex):
    for colindex in range(len(data.T)):
        col = data.T[colindex]
        hashint = bootlegHash(col)
        if hashint == 0:
            continue
        pieceStr = curfile.split('/')[-1][:-4]

        if hashint in rindex:
            if pieceStr in rindex[hashint]:
                rindex[hashint][pieceStr].append(colindex)
            else:
                rindex[hashint][pieceStr]=[colindex]
        else:
            rindex[hashint]={}
            rindex[hashint][pieceStr]=[colindex]
    return rindex

In [11]:
def N_Gram_DB(data, rindex, N_Gram = 3):
    for colindex in range(len(data.T)):
        cols = []
        try:
            for i in range(N_Gram):
                cols.append(data.T[colindex+i])
        except IndexError:
            continue
        fp = []
        equals_Zero = True
        for column in cols:
            hashint = bootlegHash(column)
            fp.append(hashint)
            if hashint != 0:
                equals_Zero = False
        if equals_Zero == True:
            continue
        fp = tuple(fp)
        pieceStr = curfile.split('/')[-1][:-4]
        if fp in rindex:
            if pieceStr in rindex[fp]:
                rindex[fp][pieceStr].append(colindex)
            else:
                rindex[fp][pieceStr]=[colindex]
        else:
            rindex[fp]={}
            rindex[fp][pieceStr]=[colindex]
    return rindex

In [30]:
def Dynamic_N_Gram_DB(data, rindex, rindex_original, threshold_function):
    for colindex in range(len(data.T)):
        first_col = data.T[colindex]
        first_fp = bootlegHash(first_col)
        hits = len(rindex_original[fp].keys())
        N_Gram = int(threshold_function(hits))
        cols = []
        try:
            for i in range(N_Gram):
                cols.append(data.T[colindex+i])
        except IndexError:
            continue
        fp = []
        equals_Zero = True
        for col in cols:
            hashint = bootlegHash(col)
            fp.append(hashint)
            if hashint != 0:
                equals_Zero = False
        if equals_Zero == True:
            continue
        fp = tuple(fp)
        pieceStr = curfile.split('/')[-1][:-4]
        if fp in rindex:
            if pieceStr in rindex[fp]:
                rindex[fp][pieceStr].append(colindex)
            else:
                rindex[fp][pieceStr]=[colindex]
        else:
            rindex[fp]={}
            rindex[fp][pieceStr]=[colindex]
    return rindex

In [ ]:
def createCountFile(outfile, rindex, N_Gram):
    rindex_count = {}
    out = "/home/dyang/SheetMidiSearchRetrieval/experiments/indices/N_GRAM_5_COUNT.pkl"
    for key in rindex:
        count = 0
        subDict = rindex[key]
        for key1 in subDict:
            count += len(subDict[key1])
        rindex_count[key] = count
    with open (outfile,"wb") as f:
        pickle.dump(rindex_count,f)
    f.close()

In [ ]:
rindex = {}
fpMap = {}
filelist = 'cfg_files/db.list'
outfile = 'experiments/indices/N_GRAM_5_ALL.pkl'
mode = "N_GRAM"
N=5
threshold_function = lambda x: 3*x/10000 + 1
with open(filelist, 'r') as f:
    failed = []
    for curfile in f:
        curfile = curfile.strip().strip('\n')
        print("Processed:", count)
        try:
            num = curfile.split('/')[-1][0]
            if(num == 'd'):
                data, _ = getTotalBscore(curfile)
            else:
                with open(curfile, 'rb') as pickle_file:
                    data = pickle.load(pickle_file)
            if mode == "SINGULAR":
                rindex = Singular_DB(data, rindex)
            elif mode == "N_GRAM":
                rindex = N_Gram_DB(data, rindex, N_Gram=N)
            elif mode == "DYNAMIC_N_GRAM":
                rindex = Dynamic_N_Gram_DB(data, rindex, rindex_original, threshold_function)
        except:
            failed.append(curfile)
    print(failed)
with open(outfile,'wb') as f:
    pickle.dump(rindex,f)
if mode == "N_GRAM":
    outfile = os.path.splitext(outfile)[0][:-3]+"COUNT.pkl"
    createCountFile(outfile, rindex, N_Gram)

Processed: 1
Processed: 2
Processed: 3
Processed: 4
Processed: 5
Processed: 6
Processed: 7
Processed: 8
Processed: 9
Processed: 10
Processed: 11
Processed: 12
Processed: 13
Processed: 14
Processed: 15
Processed: 16
Processed: 17
Processed: 18
Processed: 19
Processed: 20
Processed: 21
Processed: 22
Processed: 23
Processed: 24
Processed: 25
Processed: 26
Processed: 27
Processed: 28
Processed: 29
Processed: 30
Processed: 31
Processed: 32
Processed: 33
Processed: 34
Processed: 35
Processed: 36
Processed: 37
Processed: 38
Processed: 39
Processed: 40
Processed: 41
Processed: 42
Processed: 43
Processed: 44
Processed: 45
Processed: 46
Processed: 47
Processed: 48
Processed: 49
Processed: 50
Processed: 51
Processed: 52
Processed: 53
Processed: 54
Processed: 55
Processed: 56
Processed: 57
Processed: 58
Processed: 59
Processed: 60
Processed: 61
Processed: 62
Processed: 63
Processed: 64
Processed: 65
Processed: 66
Processed: 67
Processed: 68
Processed: 69
Processed: 70
Processed: 71
Processed: 72
P

Processed: 555
Processed: 556
Processed: 557
Processed: 558
Processed: 559
Processed: 560
Processed: 561
Processed: 562
Processed: 563
Processed: 564
Processed: 565
Processed: 566
Processed: 567
Processed: 568
Processed: 569
Processed: 570
Processed: 571
Processed: 572
Processed: 573
Processed: 574
Processed: 575
Processed: 576
Processed: 577
Processed: 578
Processed: 579
Processed: 580
Processed: 581
Processed: 582
Processed: 583
Processed: 584
Processed: 585
Processed: 586
Processed: 587
Processed: 588
Processed: 589
Processed: 590
Processed: 591
Processed: 592
Processed: 593
Processed: 594
Processed: 595
Processed: 596
Processed: 597
Processed: 598
Processed: 599
Processed: 600
Processed: 601
Processed: 602
Processed: 603
Processed: 604
Processed: 605
Processed: 606
Processed: 607
Processed: 608
Processed: 609
Processed: 610
Processed: 611
Processed: 612
Processed: 613
Processed: 614
Processed: 615
Processed: 616
Processed: 617
Processed: 618
Processed: 619
Processed: 620
Processed:

Processed: 1095
Processed: 1096
Processed: 1097
Processed: 1098
Processed: 1099
Processed: 1100
Processed: 1101
Processed: 1102
Processed: 1103
Processed: 1104
Processed: 1105
Processed: 1106
Processed: 1107
Processed: 1108
Processed: 1109
Processed: 1110
Processed: 1111
Processed: 1112
Processed: 1113
Processed: 1114
Processed: 1115
Processed: 1116
Processed: 1117
Processed: 1118
Processed: 1119
Processed: 1120
Processed: 1121
Processed: 1122
Processed: 1123
Processed: 1124
Processed: 1125
Processed: 1126
Processed: 1127
Processed: 1128
Processed: 1129
Processed: 1130
Processed: 1131
Processed: 1132
Processed: 1133
Processed: 1134
Processed: 1135
Processed: 1136
Processed: 1137
Processed: 1138
Processed: 1139
Processed: 1140
Processed: 1141
Processed: 1142
Processed: 1143
Processed: 1144
Processed: 1145
Processed: 1146
Processed: 1147
Processed: 1148
Processed: 1149
Processed: 1150
Processed: 1151
Processed: 1152
Processed: 1153
Processed: 1154
Processed: 1155
Processed: 1156
Processe

Processed: 1609
Processed: 1610
Processed: 1611
Processed: 1612
Processed: 1613
Processed: 1614
Processed: 1615
Processed: 1616
Processed: 1617
Processed: 1618
Processed: 1619
Processed: 1620
Processed: 1621
Processed: 1622
Processed: 1623
Processed: 1624
Processed: 1625
Processed: 1626
Processed: 1627
Processed: 1628
Processed: 1629
Processed: 1630
Processed: 1631
Processed: 1632
Processed: 1633
Processed: 1634
Processed: 1635
Processed: 1636
Processed: 1637
Processed: 1638
Processed: 1639
Processed: 1640
Processed: 1641
Processed: 1642
Processed: 1643
Processed: 1644
Processed: 1645
Processed: 1646
Processed: 1647
Processed: 1648
Processed: 1649
Processed: 1650
Processed: 1651
Processed: 1652
Processed: 1653
Processed: 1654
Processed: 1655
Processed: 1656
Processed: 1657
Processed: 1658
Processed: 1659
Processed: 1660
Processed: 1661
Processed: 1662
Processed: 1663
Processed: 1664
Processed: 1665
Processed: 1666
Processed: 1667
Processed: 1668
Processed: 1669
Processed: 1670
Processe

Processed: 2122
Processed: 2123
Processed: 2124
Processed: 2125
Processed: 2126
Processed: 2127
Processed: 2128
Processed: 2129
Processed: 2130
Processed: 2131
Processed: 2132
Processed: 2133
Processed: 2134
Processed: 2135
Processed: 2136
Processed: 2137
Processed: 2138
Processed: 2139
Processed: 2140
Processed: 2141
Processed: 2142
Processed: 2143
Processed: 2144
Processed: 2145
Processed: 2146
Processed: 2147
Processed: 2148
Processed: 2149
Processed: 2150
Processed: 2151
Processed: 2152
Processed: 2153
Processed: 2154
Processed: 2155
Processed: 2156
Processed: 2157
Processed: 2158
Processed: 2159
Processed: 2160
Processed: 2161
Processed: 2162
Processed: 2163
Processed: 2164
Processed: 2165
Processed: 2166
Processed: 2167
Processed: 2168
Processed: 2169
Processed: 2170
Processed: 2171
Processed: 2172
Processed: 2173
Processed: 2174
Processed: 2175
Processed: 2176
Processed: 2177
Processed: 2178
Processed: 2179
Processed: 2180
Processed: 2181
Processed: 2182
Processed: 2183
Processe

...

Processed: 28172
Processed: 28173
Processed: 28174
Processed: 28175
Processed: 28176
Processed: 28177
Processed: 28178
Processed: 28179
Processed: 28180
Processed: 28181
Processed: 28182
Processed: 28183
Processed: 28184
Processed: 28185
Processed: 28186
Processed: 28187
Processed: 28188
Processed: 28189
Processed: 28190
Processed: 28191
Processed: 28192
Processed: 28193
Processed: 28194
Processed: 28195
Processed: 28196
Processed: 28197
Processed: 28198
Processed: 28199
Processed: 28200
Processed: 28201
Processed: 28202
Processed: 28203
Processed: 28204
Processed: 28205
Processed: 28206
Processed: 28207
Processed: 28208
Processed: 28209
Processed: 28210
Processed: 28211
Processed: 28212
Processed: 28213
Processed: 28214
Processed: 28215
Processed: 28216
Processed: 28217
Processed: 28218
Processed: 28219
Processed: 28220
Processed: 28221
Processed: 28222
Processed: 28223
Processed: 28224
Processed: 28225
Processed: 28226
Processed: 28227
Processed: 28228
Processed: 28229
Processed: 282

Processed: 28657
Processed: 28658
Processed: 28659
Processed: 28660
Processed: 28661
Processed: 28662
Processed: 28663
Processed: 28664
Processed: 28665
Processed: 28666
Processed: 28667
Processed: 28668
Processed: 28669
Processed: 28670
Processed: 28671
Processed: 28672
Processed: 28673
Processed: 28674
Processed: 28675
Processed: 28676
Processed: 28677
Processed: 28678
Processed: 28679
Processed: 28680
Processed: 28681
Processed: 28682
Processed: 28683
Processed: 28684
Processed: 28685
Processed: 28686
Processed: 28687
Processed: 28688
Processed: 28689
Processed: 28690
Processed: 28691
Processed: 28692
Processed: 28693
Processed: 28694
Processed: 28695
Processed: 28696
Processed: 28697
Processed: 28698
Processed: 28699
Processed: 28700
Processed: 28701
Processed: 28702
Processed: 28703
Processed: 28704
Processed: 28705
Processed: 28706
Processed: 28707
Processed: 28708
Processed: 28709
Processed: 28710
Processed: 28711
Processed: 28712
Processed: 28713
Processed: 28714
Processed: 287

Processed: 29140
Processed: 29141
Processed: 29142
Processed: 29143
Processed: 29144
Processed: 29145
Processed: 29146
Processed: 29147
Processed: 29148
Processed: 29149
Processed: 29150
Processed: 29151
Processed: 29152
Processed: 29153
Processed: 29154
Processed: 29155
Processed: 29156
Processed: 29157
Processed: 29158
Processed: 29159
Processed: 29160
Processed: 29161
Processed: 29162
Processed: 29163
Processed: 29164
Processed: 29165
Processed: 29166
Processed: 29167
Processed: 29168
Processed: 29169
Processed: 29170
Processed: 29171
Processed: 29172
Processed: 29173
Processed: 29174
Processed: 29175
Processed: 29176
Processed: 29177
Processed: 29178
Processed: 29179
Processed: 29180
Processed: 29181
Processed: 29182
Processed: 29183
Processed: 29184
Processed: 29185
Processed: 29186
Processed: 29187
Processed: 29188
Processed: 29189
Processed: 29190
Processed: 29191
Processed: 29192
Processed: 29193
Processed: 29194
Processed: 29195
Processed: 29196
Processed: 29197
Processed: 291

Processed: 29622
Processed: 29623
Processed: 29624
Processed: 29625
Processed: 29626
Processed: 29627
Processed: 29628
Processed: 29629
Processed: 29630
Processed: 29631
Processed: 29632
Processed: 29633
Processed: 29634
Processed: 29635
Processed: 29636
Processed: 29637
Processed: 29638
Processed: 29639
Processed: 29640
Processed: 29641
Processed: 29642
Processed: 29643
Processed: 29644
Processed: 29645
Processed: 29646
Processed: 29647
Processed: 29648
Processed: 29649
Processed: 29650
Processed: 29651
Processed: 29652
Processed: 29653
Processed: 29654
Processed: 29655
Processed: 29656
Processed: 29657
Processed: 29658
Processed: 29659
Processed: 29660
Processed: 29661
Processed: 29662
Processed: 29663
Processed: 29664
Processed: 29665
Processed: 29666
Processed: 29667
Processed: 29668
Processed: 29669
Processed: 29670
Processed: 29671
Processed: 29672
Processed: 29673
Processed: 29674
Processed: 29675
Processed: 29676
Processed: 29677
Processed: 29678
Processed: 29679
Processed: 296

Processed: 30104
Processed: 30105
Processed: 30106
Processed: 30107
Processed: 30108
Processed: 30109
Processed: 30110
Processed: 30111
Processed: 30112
Processed: 30113
Processed: 30114
Processed: 30115
Processed: 30116
Processed: 30117
Processed: 30118
Processed: 30119
Processed: 30120
Processed: 30121
Processed: 30122
Processed: 30123
Processed: 30124
Processed: 30125
Processed: 30126
Processed: 30127
Processed: 30128
Processed: 30129
Processed: 30130
Processed: 30131
Processed: 30132
Processed: 30133
Processed: 30134
Processed: 30135
Processed: 30136
Processed: 30137
Processed: 30138
Processed: 30139
Processed: 30140
Processed: 30141
Processed: 30142
Processed: 30143
Processed: 30144
Processed: 30145
Processed: 30146
Processed: 30147
Processed: 30148
Processed: 30149
Processed: 30150
Processed: 30151
Processed: 30152
Processed: 30153
Processed: 30154
Processed: 30155
Processed: 30156
Processed: 30157
Processed: 30158
Processed: 30159
Processed: 30160
Processed: 30161
Processed: 301